<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Файл-сервера-xmlrpc_stats_server_p3.ipynb-на-Python-3" data-toc-modified-id="Файл-сервера-xmlrpc_stats_server_p3.ipynb-на-Python-3-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Файл сервера xmlrpc_stats_server_p3.ipynb на Python 3</a></span></li></ul></div>

#### Файл сервера xmlrpc_stats_server_p3.ipynb на Python 3 

In [ ]:
from xmlrpc.server import SimpleXMLRPCServer
from xmlrpc.server import SimpleXMLRPCRequestHandler
import datetime
import os
import csv
import glob

# Константа для максимального количества записей в журнале
MAX_LOG_SIZE = 3

class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)

server = SimpleXMLRPCServer(("localhost", 8018), 
                            requestHandler=RequestHandler)

# Проверка количества строк в файле
def check_log_size():
    if os.path.exists('log.csv'):
        with open('log.csv', 'r') as f:
            lines = f.readlines()
            return len(lines)
    return 0

# Переименование журнала с добавлением даты и времени
def rotate_log():
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    os.rename('log.csv', f'log_{timestamp}.csv')

# Добавление строки в лог с проверкой размера
def add_log(sname):
    if check_log_size() >= MAX_LOG_SIZE:
        rotate_log()
    
    # Получаем текущую дату и время
    current_time = datetime.datetime.now()

    # Форматируем строку даты/времени для записи в CSV
    formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
    
    with open('log.csv', 'a', encoding='utf-8') as f:
        f.write(f'{sname},{formatted_time}\n')
    return True

server.register_function(add_log, 'add_log')

# Функция для чтения и фильтрации логов из одного файла
def read_logs_from_file(filename, event_type=None, s_start_time=None, d_end_time=None):
    logs = []
    start_time = datetime.datetime.strptime(s_start_time, "%Y-%m-%d %H:%M:%S")
    end_time = datetime.datetime.strptime(d_end_time, "%Y-%m-%d %H:%M:%S")
    if os.path.exists(filename):
        with open(filename, 'r', encoding='utf-8') as f:
            reader = csv.reader(f)
            for row in reader:
                if len(row) != 2:
                    continue  # Игнорируем неправильные строки
                
                # Разбиваем строку: [тип события, время]
                log_event, log_time_str = row
                log_time = datetime.datetime.strptime(log_time_str, "%Y-%m-%d %H:%M:%S")
                
                # Фильтрация по типу события
                if event_type and log_event != event_type:
                    continue
                
                # Фильтрация по времени
                if start_time and log_time < start_time:
                    continue
                if end_time and log_time > end_time:
                    continue
                
                # Добавляем записи, прошедшие фильтрацию
                logs.append(row)
    return logs

# Фильтрация логов по типу операции и времени из всех файлов
def get_logs(event_type=None, start_time=None, end_time=None):
    logs = []
    
    # Получаем все файлы формата log.csv и log_YYYMMDD_hhmmss.csv
    log_files = glob.glob('log*.csv')
    
    # Читаем и фильтруем логи из каждого файла
    for log_file in log_files:
        logs.extend(read_logs_from_file(log_file, event_type, start_time, end_time))
    
    return logs

server.register_function(get_logs, 'get_logs')

print("Listening on port 8018...")
server.serve_forever()



Listening on port 8018...
